<a href="https://colab.research.google.com/github/7pupuwen/PCB_QDETECTION/blob/main/PCB_Detection_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download -d norbertelter/pcb-defect-dataset -q
!unzip -q pcb-defect-dataset.zip -d pcb_data

In [ ]:
! pip install pennylane -q

In [ ]:
import os
import pandas as pd

# Set the label files
train_labels_path = "/content/pcb_data/pcb-defect-dataset/train/labels"
val_labels_path = "/content/pcb_data/pcb-defect-dataset/val/labels"
test_labels_path = "/content/pcb_data/pcb-defect-dataset/test/labels"

# Definition of convert label to csv
def convert_labels_to_csv(labels_path, output_csv):
    data = []

    # read every .txt file
    for file in os.listdir(labels_path):
        if file.endswith(".txt"):
            file_path = os.path.join(labels_path, file)
            image_name = file.replace(".txt", ".jpg")  # The file name corresponding to the image
            with open(file_path, "r") as f:
                lines = f.readlines()  #A .txt may have multiple lines of tags

                for line in lines:
                    values = line.strip().split()
                    class_id = int(values[0])  # Defect category
                    x_center, y_center, width, height = map(float, values[1:])  # Analyze numerical values

                    # Save to DataFrame
                    data.append([image_name, class_id, x_center, y_center, width, height])

    # create DataFrame
    df = pd.DataFrame(data, columns=["image_name", "class_id", "x_center", "y_center", "width", "height"])

    # Save as CSV
    df.to_csv(output_csv, index=False)

    print(f"{output_csv} convert finish！")
    print(df.head(10))  # show the first 10 data

# covert `train`、`val`、`test` labels
convert_labels_to_csv(train_labels_path, "train_labels.csv")
convert_labels_to_csv(val_labels_path, "val_labels.csv")
convert_labels_to_csv(test_labels_path, "test_labels.csv")

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

#Set up image folder
train_images_path = "/content/pcb_data/pcb-defect-dataset/train/images"
val_images_path = "/content/pcb_data/pcb-defect-dataset/val/images"
test_images_path = "/content/pcb_data/pcb-defect-dataset/test/images"

# Read tag CSV
train_labels_csv = "train_labels.csv"
val_labels_csv = "val_labels.csv"
test_labels_csv = "test_labels.csv"

# Define functions to read images and match tags
def load_images_and_labels(images_path, labels_csv):
    df = pd.read_csv(labels_csv)  # Read CSV
    X, y = [], []  # Store images and labels

    # Read images and match tags
    for _, row in df.iterrows():
        img_path = os.path.join(images_path, row["image_name"])

        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (64, 64)) / 255.0  # standardization
            X.append(img)
            y.append(row["class_id"])  # Take only defect categories as labels

    # NumPy array
    X = np.array(X)
    y = np.array(y)

    print(f"{labels_csv} processed, image data size: {X.shape}, label data size: {y.shape}")
    return X, y

# read `train`、`val`、`test` data
X_train, y_train = load_images_and_labels(train_images_path, train_labels_csv)
X_val, y_val = load_images_and_labels(val_images_path, val_labels_csv)
X_test, y_test = load_images_and_labels(test_images_path, test_labels_csv)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# take 500 data
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=500, random_state=42, stratify=y_train)
X_val, _, y_val, _ = train_test_split(X_val, y_val, train_size=500, random_state=42, stratify=y_val)
X_test, _, y_test, _ = train_test_split(X_test, y_test, train_size=500, random_state=42, stratify=y_test)

# Confirm data shape
print("取樣後的影像數據 (Train):", X_train.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Train):", y_train.shape)  # (500,)

print("取樣後的影像數據 (Val):", X_val.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Val):", y_val.shape)  # (500,)

print("取樣後的影像數據 (Test):", X_test.shape)  # (500, 64, 64)
print("取樣後的標籤數據 (Test):", y_test.shape)  # (500,)


In [ ]:
import pennylane as qml

# Set up the quantum computer simulator (4 qubits, corresponding to 2x2 patch)
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_kernel(x):
    """
    This is a quantum circuit that converts image patches into quantum features
    """
    for i in range(n_qubits):
        qml.RY(x[i] * np.pi, wires=i)  # Embedding image data using Ry gates
        qml.Hadamard(wires=i)  # Join Hadamard Gate

    # oin Quantum Entanglement
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])

    return qml.expval(qml.PauliZ(0))  # Output quantum features

In [ ]:
def quanvolution(image):
    output_size = (image.shape[0] //2, image.shape[1] //2)
    q_image = np.zeros(output_size)

    for i in range(0, image.shape[0], 2):
        for j in range(0, image.shape[1], 2):
          patch = image[i:i+2, j:j+2].flatten()
          q_image[i//2, j//2] = quantum_kernel(patch)

    return q_image.flatten()

In [ ]:
# Do Quanvolution to data
X_train_quanv = np.array([quanvolution(img) for img in X_train])
X_val_quanv = np.array([quanvolution(img) for img in X_val])
X_test_quanv = np.array([quanvolution(img) for img in X_test])

print("Quantum feature size（Train）:", X_train_quanv.shape)
print("Quantum feature size（Val）:", X_val_quanv.shape)
print("Quantum feature size（Test）:", X_test_quanv.shape)


In [ ]:
# Quantum Kernel Matrix
train_kernel_matrix = compute_kernel_matrix(X_train_quanv)
val_kernel_matrix = np.array([[np.dot(x1, x2) for x2 in X_train_quanv] for x1 in X_val_quanv])
test_kernel_matrix = np.array([[np.dot(x1, x2) for x2 in X_train_quanv] for x1 in X_test_quanv])

print("Train Kernel Matrix:", train_kernel_matrix.shape)  # (N_train, N_train)
print("Val Kernel Matrix:", val_kernel_matrix.shape)      # (N_val, N_train)
print("Test Kernel Matrix:", test_kernel_matrix.shape)    # (N_test, N_train)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# train Quantum SVM
qsvm = svm.SVC(kernel="precomputed")
qsvm.fit(train_kernel_matrix, y_train)


In [ ]:
# test  QSVM in val set
preds_val = qsvm.predict(val_kernel_matrix)
accuracy_val = accuracy_score(y_val, preds_val)
print(f"QSVM Validation Accuracy: {accuracy_val:.4f}")

# test QSVM in test set
preds_test = qsvm.predict(test_kernel_matrix)
accuracy_test = accuracy_score(y_test, preds_test)
print(f"QSVM Test Accuracy: {accuracy_test:.4f}")
